### spare code
useful at the time, but not any more, may be useful in the future

In [ ]:
# so if we have an image with multiple things inside, we just want the biggest crystal -> want to return the index of these, so can just use it to extract it
# Convert the dataframe to a binary image where 0 is True (activated) and all else is False
binary_image = (one_crystal == 0)

# Label the connected components
labeled_image, num_features = label(binary_image)

plt.imshow(binary_image, cmap='gray')

props = regionprops(labeled_image)
if props:
    # Find the component with the maximum area
    largest_component = max(props, key=lambda x: x.area)
    print(largest_component)

    # Extract x-values (column indices) of the largest component
    coords = largest_component.coords
    x_values = np.unique(coords[:, 1])  # column indices are the x-values
else:
    x_values = np.array([])  # No components found

In [ ]:
## functions for particle calculations
## Min + Max dimensions calculation - Feret Diameters
# found other thing worked better

def calculate_feret_diameters(points):
    """Calculate both minimum and maximum Feret diameters using the convex hull and rotating calipers."""
    # Check if all points are collinear
    if np.linalg.matrix_rank(points - points[0]) < 2:
        # Calculate maximum and minimum distances between points as they are on a line
        dists = distance_matrix(points, points)
        max_feret = np.max(dists)
        min_feret = np.min(dists[dists > 0])  # Exclude zero distance (self to self)
        return min_feret, max_feret
        
    hull = ConvexHull(points)
    hull_points = points[hull.vertices]
    
    min_diameter = np.inf  # Start with infinitely large diameter
    max_diameter = 0  # Start with zero diameter
    num_vertices = len(hull_points)

    for i in range(num_vertices):
        j = (i + 1) % num_vertices
        p1, p2 = hull_points[i], hull_points[j]
        # Vector from p1 to p2
        edge_vec = p2 - p1
        edge_length = np.linalg.norm(edge_vec)
        max_diameter = max(max_diameter, edge_length)  # Maximum Feret diameter
        
        # Normalize the edge vector
        edge_vec /= edge_length
        # Project points onto the line normal to the edge
        normal_vec = np.array([-edge_vec[1], edge_vec[0]])
        projections = np.dot(hull_points - p1, normal_vec)
        min_diameter = min(min_diameter, max(projections) - min(projections))  # Minimum Feret diameter
    
    return min_diameter, max_diameter

def feret_diameters(contours):
    if contours:
        # Assume the largest contour is the main one (if there are multiple)
        largest_contour = max(contours, key=lambda x: x.shape[0])
        # Using convex hull to calculate Feret diameters
        hull = ConvexHull(largest_contour)
        hull_points = largest_contour[hull.vertices]
    
        # Calculate max Feret diameter
        dists = distance_matrix(hull_points, hull_points)
        max_feret = np.max(dists)
    
        # Calculate min Feret diameter
        min_feret = np.inf
        num_vertices = len(hull_points)
        for i in range(num_vertices):
            j = (i + 1) % num_vertices
            p1, p2 = hull_points[i], hull_points[j]
            edge_vec = p2 - p1
            edge_length = np.linalg.norm(edge_vec)
            edge_vec /= edge_length
            normal_vec = np.array([-edge_vec[1], edge_vec[0]])
            projections = np.dot(hull_points - p1, normal_vec)
            min_feret = min(min_feret, max(projections) - min(projections))

        return min_feret, max_feret

def calculate_feret_diameter(coords):
    """Calculate the Feret diameter for a set of points."""
    dist_matrix = distance_matrix(coords, coords)
    feret_diameter = np.max(dist_matrix)
    return feret_diameter

### ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
def calculate_properties(crystal):
    dat = {}
    #thresh = threshold(crystal.astype('uint8'), 0, 255, THRESH_BINARY + THRESH_OTSU)

    # Optionally fill holes
    thresh = binary_fill_holes(crystal).astype(int)
    contours = measure.find_contours(thresh, 0.5)
    # Sort contours by area (largest first) and select the largest contour
    contour = max(contours, key=lambda x: x.shape[0])
    
    # Label the image based on the threshold
    labeled_image = measure.label(thresh)
    
    # Extract properties using regionprops
    region = measure.regionprops(labeled_image)[0]  # Assumes largest labeled region corresponds to largest contour
    
    # Store the extracted properties in the data structure
    dat['len'] = region.major_axis_length
    dat['wid'] = region.minor_axis_length
    dat['area'] = region.area
    dat['orientation'] = region.orientation
    dat['centroid'] = region.centroid

    return dat

In [ ]:
## old when extracting data from individual crystals
## this was within the loop
multi[particle.label] = [coords[0][0],coords[-1][0],s_idx,e_idx] # y1,y2,x1,x2 
                # x values are our index also needed for extracting time
                # i.e. [f2ds['ImageData'][y1:y2,x1:x2]


keys = list(multi.keys())
extract = f2ds['ImageData'][multi[keys[0]][0]-2:multi[keys[0]][1]+3,multi[keys[0]][2] -2:multi[keys[0]][3]+3] ## added 2 pixel buffer around single crystal
start_t = time_xr['utc_time'][multi[keys[0]][2]]
end_t = time_xr['utc_time'][multi[keys[0]][3]]